In [843]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import FeatureEngineering
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from joblib import Parallel, delayed
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, recall_score, precision_score
from sklearn.metrics import roc_auc_score 
from sklearn.compose import ColumnTransformer

In [713]:
dataset = FeatureEngineering.createData(15.00)
X = dataset.iloc[:,3:].drop(columns=['final_result'])
y = dataset['final_result']
mapping = {'Pass': 0, 'Withdrawn': 1, 'Distinction': 2}
y_encoded = y.map(mapping)

In [752]:
models = [
    RandomForestClassifier(n_estimators=500,random_state=42),
    GradientBoostingClassifier(n_estimators=500,random_state=43),
    SVC(probability=True,random_state=44),
    LogisticRegression(random_state=45)
]

def evaluate_model(model,X,y,preprocessor,stratified_kfold,scoring):
    pipeline = make_pipeline(preprocessor,model)
    return cross_validate(pipeline,X,y,cv=stratified_kfold,scoring=scoring,return_train_score=True)

preprocessor = FeatureEngineering.create_preprocessor(X)
scoring = {'precision_class_1':make_scorer(precision_score,labels=[1],average=None),
          'recall_class_1':make_scorer(recall_score,labels=[1],average=None)}
stratified_kfold = StratifiedKFold(n_splits=4)

cross_val_scores = Parallel(n_jobs=4)(
    delayed(evaluate_model)(model,X,y_encoded, preprocessor,stratified_kfold,scoring) for model in models
)

In [772]:
cross_val_result_df = pd.concat(
    [pd.concat([pd.DataFrame(cross_val_scores[0]),pd.Series(["Random Forrest Classifier"]*4,name='model')],axis=1),
     pd.concat([pd.DataFrame(cross_val_scores[1]),pd.Series(["Gradient Boosting Classifier"]*4,name='model')],axis=1),
     pd.concat([pd.DataFrame(cross_val_scores[2]),pd.Series(["SVC"]*4,name='model')],axis=1) ,
     pd.concat([pd.DataFrame(cross_val_scores[3]),pd.Series(["Logistic Regression"]*4,name='model')],axis=1)
    ],axis=0
)
cross_val_result_df

,fit_time,score_time,test_precision_class_1,train_precision_class_1,test_recall_class_1,train_recall_class_1,model
0,62.284590,1.048157,0.614351,1.000000,0.742574,1.000000,Random Forrest Classifier
1,56.796354,1.734053,0.479306,0.999868,0.568713,1.000000,Random Forrest Classifier
2,57.400762,1.103153,0.580828,1.000000,0.527921,0.999868,Random Forrest Classifier
3,56.922791,1.648499,0.592202,1.000000,0.511287,0.999868,Random Forrest Classifier
0,374.185276,0.333915,0.585572,0.823671,0.794059,0.723960,Gradient Boosting Classifier
1,303.480785,0.215219,0.500850,0.858888,0.583366,0.742442,Gradient Boosting Classifier
2,325.499863,0.330402,0.579899,0.845522,0.546139,0.734125,Gradient Boosting Classifier
3,343.245556,0.361990,0.596093,0.840776,0.519604,0.738218,Gradient Boosting Classifier
0,486.317586,25.828110,0.663838,0.813507,0.712475,0.659934,SVC
1,461.946244,29.405070,0.533135,0.854985,0.497030,0.657690,SVC


In [716]:
dataset = FeatureEngineering.createData(15.00)
X = dataset.iloc[:,3:].drop(columns=['final_result'])
y = dataset['final_result']
mapping = {'Pass': 0, 'Withdrawn': 1, 'Distinction': 2}
y_encoded = y.map(mapping)
preprocessor = FeatureEngineering.create_preprocessor(X) # column transformer 
X_train, X_test, y_train, y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42,stratify=y_encoded) # for train and evaluate

def train_and_evaluate(model):
    global preprocessor
    global X_train, X_test, y_train, y_test
    pipeline = make_pipeline(preprocessor,model) 
    pipeline.fit(X_train,y_train)
    y_pred = pipeline.predict(X_test)
    precision = precision_score(y_test,y_pred,labels=[1],average='weighted')
    recall = recall_score(y_test,y_pred,labels=[1],average='weighted')
    accuracy = accuracy_score(y_test,y_pred)
    probabilities = pipeline.predict_proba(X_test)
    
    result_dictionary = {
        'model_name':model.__class__.__name__,
        'precision':precision,
        'recall':recall,
        'accuracy':accuracy,
        'y_probabilities':pd.DataFrame(probabilities)
    }
    
    return result_dictionary

In [717]:
models = [
    RandomForestClassifier(n_estimators=500,random_state=42),
    GradientBoostingClassifier(n_estimators=500,random_state=42),
    SVC(probability=True,random_state=42),
    LogisticRegression(random_state=42)
]

In [718]:
result = Parallel(n_jobs=4)(
    delayed(train_and_evaluate)(model) for model in models
)

In [719]:
result

[{'model_name': 'RandomForestClassifier',
  'precision': 0.7818877551020408,
  'recall': 0.6069306930693069,
  'accuracy': 0.665553809897879,
  'y_probabilities':           0      1      2
  0     0.626  0.274  0.100
  1     0.356  0.618  0.026
  2     0.608  0.242  0.150
  3     0.022  0.974  0.004
  4     0.798  0.050  0.152
  ...     ...    ...    ...
  5087  0.622  0.130  0.248
  5088  0.626  0.242  0.132
  5089  0.000  0.996  0.004
  5090  0.604  0.120  0.276
  5091  0.652  0.204  0.144
  
  [5092 rows x 3 columns]},
 {'model_name': 'GradientBoostingClassifier',
  'precision': 0.7684583579444773,
  'recall': 0.6440594059405941,
  'accuracy': 0.6704634721131186,
  'y_probabilities':              0         1         2
  0     0.719002  0.189257  0.091741
  1     0.156238  0.824491  0.019271
  2     0.480536  0.392100  0.127364
  3     0.038048  0.954280  0.007672
  4     0.644640  0.102843  0.252516
  ...        ...       ...       ...
  5087  0.484410  0.148607  0.366983
  5088  0.

In [721]:
for model_result in result:
    print(f"{model_result['model_name']}: Precision - {model_result['precision']}, Recall - {model_result['recall']}, Accuracy - {model_result['accuracy']}")

RandomForestClassifier: Precision - 0.7818877551020408, Recall - 0.6069306930693069, Accuracy - 0.665553809897879
GradientBoostingClassifier: Precision - 0.7684583579444773, Recall - 0.6440594059405941, Accuracy - 0.6704634721131186
SVC: Precision - 0.8044206296048225, Recall - 0.5945544554455445, Accuracy - 0.6694815396700707
LogisticRegression: Precision - 0.7554766133806986, Recall - 0.6316831683168317, Accuracy - 0.6626080125687352


In [722]:
model_scores = pd.DataFrame(result)

In [723]:
list_of_probs = []
for model_result in result:
    name = model_result['model_name']
    y_pred_and_true = np.concatenate((model_result['y_probabilities'],y_test.to_numpy().reshape(-1,1)),axis=-1)
    list_of_probs.append(y_pred_and_true)

In [724]:
list_of_probs

[array([[0.626, 0.274, 0.1  , 0.   ],
        [0.356, 0.618, 0.026, 1.   ],
        [0.608, 0.242, 0.15 , 2.   ],
        ...,
        [0.   , 0.996, 0.004, 1.   ],
        [0.604, 0.12 , 0.276, 0.   ],
        [0.652, 0.204, 0.144, 0.   ]]),
 array([[0.71900209, 0.18925671, 0.0917412 , 0.        ],
        [0.15623802, 0.82449133, 0.01927065, 1.        ],
        [0.48053593, 0.39209978, 0.12736429, 2.        ],
        ...,
        [0.03310112, 0.9579582 , 0.00894069, 1.        ],
        [0.71899046, 0.08381937, 0.19719017, 0.        ],
        [0.7270894 , 0.14308394, 0.12982667, 0.        ]]),
 array([[0.7395317 , 0.14021354, 0.12025475, 0.        ],
        [0.11340542, 0.85436032, 0.03223426, 1.        ],
        [0.62579274, 0.27129882, 0.10290843, 2.        ],
        ...,
        [0.08381547, 0.87055769, 0.04562685, 1.        ],
        [0.67415149, 0.16432652, 0.16152199, 0.        ],
        [0.69887032, 0.19816548, 0.1029642 , 0.        ]]),
 array([[0.6469027 , 0.20391011

In [725]:
def get_auc_score(y_pred_and_true,class1,class2): 
    # class1 = 0, class2 = 1
    
    df = pd.DataFrame(y_pred_and_true, columns=['prob_class_0', 'prob_class_1', 'prob_class_2', 'true_class'])

    # filter observacija tako da uključuju samo klase 0 i 1 
    binary_df = df[df['true_class'].isin([class1, class2])] 

    # ekstrakcija stvarnih labela
    true_labels_binary = binary_df['true_class']
    
    # ekstrakcija predvidjenih verovatnoća za versus klasu tj klasu 2 (jer poredimo class1 vs class2)
    if class2 == 1:
        predicted_probs_binary = binary_df['prob_class_1']
    elif class2 == 0:
        predicted_probs_binary = binary_df['prob_class_0']
    else:
        predicted_probs_binary = binary_df['prob_class_2']

    if class2 < class1: # class2 = 0 and class1 = 1
        # da bismo osigurali konzistentnost računanja AUC-a.
        # ako je class1 = 0 class2 = 1 tj. class1<class0, po difoltu ce class1 biti uzeta kao negativna klasa a class2 kao pozitivna klasa 
        auc = roc_auc_score(1 - true_labels_binary, predicted_probs_binary) # 1 - true_labels_binary zato sto sada ciljamo drugu klasu 
    else:
        auc = roc_auc_score(true_labels_binary, predicted_probs_binary) # 0.8024634563944455 salju se 0 i 1 true, a 
    # auc10 = roc_auc_score(1-true_labels_binary,predicted_probs_binary2)
    
    return auc

In [726]:
#{'Pass': 0, 'Withdrawn': 1, 'Distinction': 2}
auc01 = get_auc_score(y_pred_and_true,0,1) # koliko dobro hvatam 1 u nulama i jedinicama 
# koliko sam puta predivideo da je bila jedinica kad je stvarno bila jedinica u odnosu na to koliko sam puta predvideo da je 1 kad je bila 0? 
auc02 = get_auc_score(y_pred_and_true,0,2) # koliko dobro hvatam 2 u nulama i dvojakama
auc10 = get_auc_score(y_pred_and_true,1,0) # koliko dobro hvatam 0 u nulama i jedinicama
auc20 = get_auc_score(y_pred_and_true,2,0) # koliko dobro hvatam 0 u nulama i dvojakama 0.4884847815351025 !!! model ne zna da prepozna pass kategoriju medju onima koji su distinction. 
auc12 = get_auc_score(y_pred_and_true,1,2) 
auc21 = get_auc_score(y_pred_and_true,2,1)
sum = auc01+auc10+auc20+auc02+auc12+auc21
auc20     

0.5002884605064024

In [727]:
sum/(3*2) # prosecan auc !!!

0.7516258248743933

In [728]:
def get_scores_for_model(y_pred_and_true):
    auc_scores = []
    class_labels = [0,1,2]
    for i in range(len(class_labels)):
        for j in range(len(class_labels)):
            if class_labels[i]!= class_labels[j]:
                result_auc = get_auc_score(y_pred_and_true,class_labels[i],class_labels[j])
                result_dictionary = {
                    'AUC':result_auc,
                    'target_class':class_labels[j],
                    'versus_class':class_labels[i]
                }
                auc_scores.append(result_dictionary) 
    return auc_scores

In [729]:
final_list = []
for probs in list_of_probs:
    result_element = get_scores_for_model(probs)
    final_list.append(result_element)
rfc_df = pd.DataFrame(final_list[0])
rfc_df['model'] = ['Random Forrest Classifier']*6
gbc_df = pd.DataFrame(final_list[1])
gbc_df['model'] = ['Gradient Boosting Classifier']*6
svc_df = pd.DataFrame(final_list[2])
svc_df['model'] = ['SVC']*6
lr_df = pd.DataFrame(final_list[3])
lr_df['model'] = ['Logistic Regression']*6
metrics_df = pd.concat([rfc_df,gbc_df, svc_df, lr_df],axis=0)
metrics_df

,AUC,target_class,versus_class,model
0,0.830692,1,0,Random Forrest Classifier
1,0.670268,2,0,Random Forrest Classifier
2,0.803083,0,1,Random Forrest Classifier
3,0.879455,2,1,Random Forrest Classifier
4,0.515107,0,2,Random Forrest Classifier
5,0.891772,1,2,Random Forrest Classifier
0,0.830236,1,0,Gradient Boosting Classifier
1,0.689264,2,0,Gradient Boosting Classifier
2,0.803464,0,1,Gradient Boosting Classifier
3,0.891943,2,1,Gradient Boosting Classifier
